In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('./immo_data.csv')
print(df.shape)
print(df.columns)

In [ ]:
df_2 = df.loc[:,['scoutId', 'regio1', 'regio2', 'regio3', 'geo_plz', 'heatingType', 'newlyConst', 'balcony', 'yearConstructed', 'noParkSpaces', 'hasKitchen', 'baseRent', 'livingSpace', 'condition', 'interiorQual', 'petsAllowed', 'lift', 'typeOfFlat', 'noRooms', 'garden', 'facilities', 'lastRefurbish']]
df_2 = df_2[df_2.baseRent.between(50,10000, inclusive='neither')]
df_2.head(5)

In [ ]:

y = np.log(df_2['baseRent'])
x=df_2.copy()
x.drop(['baseRent'],axis=1,inplace = True)
x.head(10)

In [ ]:
print(df_2["condition"].unique())
print(df_2["interiorQual"].unique())
print(df_2["typeOfFlat"].unique())
print(df_2["facilities"].unique())

In [ ]:
categories = {
  "regio1": [list(np.sort(df_2['regio1'].unique()))],
  "regio2": [list(np.sort(df_2['regio2'].unique()))],
  "regio3": [list(np.sort(df_2['regio3'].unique()))],
  "heatingType": [list(np.sort(df_2['heatingType'].unique()))],
  "condition": [list(np.sort(df_2['condition'].unique()))],
  "interiorQual": [list(np.sort(df_2['interiorQual'].unique()))],
  "typeOfFlat": [list(np.sort(df_2['typeOfFlat'].unique()))],
  "facilities": [list(np.sort(df_2['facilities'].unique()))]
}

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

preprocessor = ColumnTransformer(
    [
        ('onehot_regio1', OneHotEncoder(categories=categories["regio1"]), [df_2.index('regio1')]),  # data_columns.bla -> column_name?
        ('onehot_regio2', OneHotEncoder(categories=categories["regio2"]), [df_2.index('regio2')]), # data_columns.bla -> column_name?
        ('onehot_regio3', OneHotEncoder(categories=categories["regio3"]), [df_2.index('regio3')]), # data_columns.bla -> column_name?
        ('onehot_heating_type', OneHotEncoder(categories=categories["heatingType"]), [df_2.index('heatingType')]), # data_columns.bla -> column_name?
        ('onehot_condition', OneHotEncoder(categories=categories["condition"]), [df_2.index('condition')]), # data_columns.bla -> column_name?
        ('onehot_interiorQual', OneHotEncoder(categories=categories["interiorQual"]), [df_2.index('interiorQual')]), # data_columns.bla -> column_name?
        ('onehot_typeOfFlat', OneHotEncoder(categories=categories["typeOfFlat"]), [df_2.index('typeOfFlat')]), # data_columns.bla -> column_name?
        ('onehot_facilities', OneHotEncoder(categories=categories["facilities"]), [df_2.index('facilities')]), # data_columns.bla -> column_name?
        ('scaler', StandardScaler(), [data_columns.index(col) for col in standard_scaler_col_list])
    ],
    remainder='drop'
)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, 
                                                    random_state=1)
X_train

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

#Best hyperparameters from Random Search:
#maxdepth: 16, minsamleaf: 117, n: 73, maxfeat: 10, lr: 0.07
def gradientboostingmachine(md, msl, n, mf, lr, xtrain, ytrain, xtest, ytest):
    gbm_best = GradientBoostingRegressor(n_estimators=n, random_state=1111,
                                         max_depth=md, max_features=mf, 
                                         min_samples_leaf=msl, learning_rate=lr
                                         )
    gbm_best.fit(xtrain, ytrain)
    y_pred_gbm = gbm_best.predict(xtest)
    
        
    #Saving the model
    Pkl_Filename = "GB_Model.pkl"  
    with open(Pkl_Filename, 'wb') as file:  
        pickle.dump(gbm_best, file)
        
    print('MAE:', metrics.mean_absolute_error(ytest, y_pred_gbm))
    print('MSE:', metrics.mean_squared_error(ytest, y_pred_gbm))
    print('R2_score:', metrics.r2_score(ytest, y_pred_gbm))

    plt.scatter(y_pred_gbm,ytest)
    plt.xlabel('predicted value of y')
    plt.ylabel('y')
    plt.figure()
gradientboostingmachine(16, 117, 73, 10, 0.07, X_train, y_train, X_test, y_test) 